## Phase 1: Data extraction

https://news.ycombinator.com/item?id=41614126

https://github.com/conjuncts/gmft

https://www.reddit.com/r/LocalLLaMA/comments/1854d06/table_extraction_from_pdf/ ==> some guy did https://docs.google.com/spreadsheets/d/12IhxHZbYF71dPl32PQpF_6pg9e9S8f9W4sTHt-B0KTg/edit?gid=0#gid=0 


https://www.getmagical.com/blog/extract-table-from-pdf


As to whether traditional OCR or LLMs are better for this task

#### Conclusions:
- We REALLY want to keep the right figures, so potentially trying lower end models for this, while it can be accurate a fair amount of the time, likely does not counteract the risk of it eventually hallucinating. Not all LLMs are equal.
- [Marker](https://github.com/vikParuchuri/marker) has 20k stars on github and was found from the excel sheet; but has a commercial usage agreement so Nahh

https://github.com/conjuncts/gmft ? => Relies on Table transformers (microsoft), small, seems ideal. I've also seen stuff for AWS textract but let's keep it simple for now.

In [1]:
import gmft
from gmft.auto import AutoTableDetector, AutoTableFormatter
from gmft.pdf_bindings import PyPDFium2Document
import pandas as pd

def extract_tables_from_pdf(pdf_path):
    """Extracts tables from a PDF file and converts them to Pandas DataFrames."""
    detector = AutoTableDetector()
    formatter = AutoTableFormatter()
    
    doc = PyPDFium2Document(pdf_path)
    tables = []
    
    for page in doc:
        extracted_tables = detector.extract(page)
        for table in extracted_tables:
            print(table)
            df = formatter.format(table).to_pandas()
            tables.append(df)
    
    doc.close()
    return tables

def save_tables_as_csv(tables, base_filename):
    """Saves extracted tables to CSV files."""
    for i, df in enumerate(tables):
        filename = f"{base_filename}_table_{i+1}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved: {filename}")

if __name__ == "__main__":
    pdf_path = "./fwc_sample_financial_statement 1.pdf"
    tables = extract_tables_from_pdf(pdf_path)
    save_tables_as_csv(tables, "financial_statement")


k:\conda_envs\commbank_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'TATRFormattedTable' object has no attribute 'to_pandas'